# Social RL Demo: Learning Through Interaction

This notebook demonstrates the Social RL architecture - a novel approach to agent learning that uses social dynamics as the reward signal.

## Key Concepts

| Traditional RL | Social RL |
|---------------|----------|
| Environment | Other agents + theoretical constraints |
| State | Round context + concept manifestations |
| Action | Agent utterance/response |
| Reward | Social feedback (engagement, alignment, contribution) |
| Policy | PRAR process schemas |

## Components

1. **ContextInjector**: Dynamic manifestation generation per turn
2. **SocialFeedbackExtractor**: Extract learning signals from interaction
3. **ProcessRetriever**: PRAR-based reasoning guidance
4. **SocialRLRunner**: Main execution engine

## 1. Environment Setup

In [ ]:
# Setup paths
import sys
import os

# For Colab
if 'google.colab' in sys.modules:
    if not os.path.exists('/content/Socratic-RCM'):
        !git clone https://github.com/Baglecake/Socratic-RCM.git /content/Socratic-RCM
    os.chdir('/content/Socratic-RCM')
    BASE_PATH = '/content/Socratic-RCM'
else:
    # Local - go up one level from notebooks/
    BASE_PATH = os.path.dirname(os.getcwd())
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
        BASE_PATH = os.getcwd()

# Add module paths with absolute paths
sys.path.insert(0, os.path.join(BASE_PATH, 'social_rl'))
sys.path.insert(0, os.path.join(BASE_PATH, 'agents'))
sys.path.insert(0, os.path.join(BASE_PATH, 'local_rcm'))

print(f"Base path: {BASE_PATH}")
print(f"Working directory: {os.getcwd()}")
print(f"social_rl exists: {os.path.exists(os.path.join(BASE_PATH, 'social_rl'))}")

## 2. Test Individual Components

In [ ]:
# Test ContextInjector
from context_injector import (
    ContextInjector, TheoreticalFramework, ManifestationType
)

# Create framework for Option A (Class Conflict / Alienation)
framework = TheoreticalFramework(
    option="A",
    label="Class Conflict / Alienation",
    concept_a_name="Alienation",
    concept_a_definition="Workers become separated from the products of their labor",
    concept_b_name="Non-domination",
    concept_b_definition="Freedom from arbitrary power"
)

injector = ContextInjector(framework, mode=ManifestationType.PROGRESSIVE)
print("ContextInjector initialized with PROGRESSIVE mode")

In [ ]:
# Test manifestation generation at different intensities
test_agent = {
    "identifier": "Worker+Alice",
    "role": "Worker",
    "name": "Alice",
    "goal": "Gain influence over work organization",
    "persona": "Thoughtful but hesitant, often suppressing ideas"
}

test_round = {
    "round_number": 1,
    "scenario": "Manufacturing workshop - Baseline alienation conditions",
    "rules": "Workers CAN complete tasks. Workers CANNOT suggest changes.",
    "tasks": "Complete three production cycles",
    "concept_a_manifestation": "Alienation manifests as separation from decision-making",
    "concept_b_manifestation": "Non-domination is absent - arbitrary power unchecked",
    "end_condition": "Total messages: 15"
}

# Generate contexts at turns 1, 7, and 14 (low, medium, high intensity)
for turn in [1, 7, 14]:
    context = injector.generate_turn_context(
        agent_id="Worker+Alice",
        agent_config=test_agent,
        round_config=test_round,
        turn_number=turn,
        conversation_history=[]
    )
    intensity = injector._calculate_intensity(turn, 15)
    print(f"\n=== Turn {turn} (intensity: {intensity}) ===")
    print(f"Experiential: {context.experiential_context}")
    print(f"PRAR Cue: {context.prar_cue[:100]}...")

In [ ]:
# Test SocialFeedbackExtractor
from feedback_extractor import SocialFeedbackExtractor, ConceptMarkers

extractor = SocialFeedbackExtractor(ConceptMarkers.for_option_a())

# Simulate a conversation
test_messages = [
    {"agent_id": "Owner+Marta", "content": "Alice, you will work on the assembly line today. No questions."},
    {"agent_id": "Worker+Alice", "content": "I understand, Marta. Though I don't really understand why we changed the process."},
    {"agent_id": "Worker+Ben", "content": "Alice, did you hear about the new quotas? They've increased again."},
    {"agent_id": "Worker+Alice", "content": "Yes, Ben. It feels meaningless - we just follow orders without any say."},
    {"agent_id": "Owner+Marta", "content": "Less talking, more working. Ben, you must complete your station."},
    {"agent_id": "Worker+Ben", "content": "Yes, okay. I'll get to it right away."},
]

feedback = extractor.extract_round_feedback(
    round_number=1,
    messages=test_messages,
    participants=["Owner+Marta", "Worker+Alice", "Worker+Ben"]
)

print(extractor.generate_feedback_report(1))

In [ ]:
# Test ProcessRetriever
from process_retriever import ProcessRetriever

retriever = ProcessRetriever(framework_option="A")

# Test basic policy retrieval
worker_policy = retriever.retrieve_policy("Worker", round_number=1, turn_number=1)
print(f"Worker Policy: {worker_policy.name}")
print(f"\nCompiled Policy:\n{worker_policy.compile()}")

# Test with low engagement (should trigger challenge activation)
print("\n--- With Low Engagement Feedback ---")
low_feedback = {"engagement": 0.2, "theoretical_alignment": 0.6}
adapted = retriever.retrieve_policy("Worker", feedback=low_feedback, round_number=2, turn_number=5)
print(f"Adapted Policy: {adapted.name}")
print(f"\nRCM Cue:\n{retriever.generate_rcm_cue(adapted, low_feedback)}")

## 3. Load Canvas and Create Full Runner

In [ ]:
import json
from pathlib import Path

# Find state file
state_paths = [
    'prar/outputs/2025-11-23_baseline_full_qwen/state.json',
    '/content/Socratic-RCM/prar/outputs/2025-11-23_baseline_full_qwen/state.json'
]

state_path = None
for p in state_paths:
    if Path(p).exists():
        state_path = p
        break

if state_path:
    with open(state_path, 'r') as f:
        state = json.load(f)
    canvas = state['canvas']
    print(f"Loaded canvas from: {state_path}")
    print(f"Framework: {canvas['project'].get('theoretical_option_label')}")
    print(f"Agents: {[a['identifier'] for a in canvas['agents']]}")
    print(f"Rounds: {len(canvas['rounds'])}")
else:
    print("State file not found. Run baseline experiment first.")

In [ ]:
# Create a Mock LLM for testing (replace with real LLM for actual runs)
class MockLLMClient:
    """Mock LLM that generates role-appropriate responses."""
    def __init__(self):
        self.call_count = 0
        self.responses = {
            "Worker": [
                "I understand. I'll get started on the task right away.",
                "The work continues. I'm not sure why we do it this way, but I follow the process.",
                "Another cycle complete. It feels... mechanical.",
                "Yes, I'll handle that. Though I wonder if there's a better approach."
            ],
            "Owner": [
                "Good. Let's proceed with the schedule as planned. No delays.",
                "I need the assembly line running at full capacity. Focus on your tasks.",
                "The quotas are set. Meet them.",
                "Less discussion, more production. That's what matters here."
            ],
            "Analyst": [
                "Observing the interaction patterns: workers show compliance with limited agency.",
                "The dialogue reveals markers of alienation - task execution without meaning.",
                "Authority is exercised through directive communication. Workers acknowledge but don't engage."
            ]
        }
    
    def send_message(self, system_prompt: str, user_message: str) -> str:
        self.call_count += 1
        
        # Detect role from system prompt
        role = "Worker"  # default
        if "Owner" in system_prompt:
            role = "Owner"
        elif "Analyst" in system_prompt:
            role = "Analyst"
        
        responses = self.responses[role]
        return responses[self.call_count % len(responses)]

mock_llm = MockLLMClient()
print("Mock LLM client created")

In [ ]:
# Create SocialRLRunner
from runner import SocialRLRunner, SocialRLConfig

config = SocialRLConfig(
    manifestation_mode="progressive",  # Options: static, progressive, reactive, adaptive
    use_prar_cues=True,
    prar_intensity="medium",
    use_coach_validation=False,  # Disable for mock LLM
    verbose=True
)

if state_path:
    runner = SocialRLRunner(canvas, mock_llm, config)
    print("\nSocialRLRunner created successfully!")

## 4. Execute Social RL Round

In [ ]:
# Execute Round 1 with limited turns for demo
if state_path:
    result = runner.execute_round(round_number=1, max_turns=6)
    print(f"\nRound complete: {len(result.messages)} messages")

In [ ]:
# Examine a message with Social RL metadata
if state_path and result.messages:
    msg = result.messages[1]  # Second message
    print(f"=== Message Analysis ===")
    print(f"Agent: {msg.agent_id}")
    print(f"Turn: {msg.turn_number}")
    print(f"Content: {msg.content}")
    print(f"\nPRAR Cue Used:")
    print(msg.prar_cue_used if msg.prar_cue_used else "(none)")
    print(f"\nFeedback Snapshot: {msg.feedback_snapshot}")

In [ ]:
# View feedback for round
if state_path:
    for agent_id, fb in result.feedback.items():
        print(f"\n{agent_id}:")
        print(f"  Engagement: {fb.engagement:.2f}")
        print(f"  Theoretical Alignment: {fb.theoretical_alignment:.2f}")
        print(f"  Contribution Value: {fb.contribution_value:.2f}")
        print(f"  Concepts Embodied: {list(set(fb.concepts_embodied))}")

## 5. Multi-Round Learning Demo

In [ ]:
# Execute Round 2 and observe policy adaptation
if state_path:
    print("\n" + "="*60)
    print("ROUND 2 - With policy adaptation from Round 1 feedback")
    print("="*60)
    
    result2 = runner.execute_round(round_number=2, max_turns=6)
    
    if result2.policy_adaptations:
        print("\nPolicy Adaptations Made:")
        for adapt in result2.policy_adaptations:
            print(f"  {adapt['agent_id']}: {adapt['deltas']}")

In [ ]:
# Compare feedback across rounds
if state_path:
    comparison = runner.feedback_extractor.compare_rounds(1, 2)
    print("=== Feedback Delta (Round 1 -> Round 2) ===")
    for agent_id, deltas in comparison.items():
        print(f"\n{agent_id}:")
        for signal, delta in deltas.items():
            direction = "+" if delta > 0 else ""
            print(f"  {signal}: {direction}{delta:.3f}")

In [ ]:
# Generate final report
if state_path:
    print(runner.generate_report())

## 6. Key Insights

### What Makes This "Social RL"?

1. **No Explicit Reward Function**: Learning signals emerge from interaction
   - Being referenced = engagement signal
   - Embodying concepts = alignment signal
   - Inclusion in synthesis = contribution signal

2. **Process Retrieval as Policy**: PRAR guides HOW to reason, not WHAT to say
   - Policies adapt based on feedback
   - No model weight updates needed

3. **Dynamic Context Injection**: Each turn gets fresh manifestations
   - Intensity scales through the round
   - Context reacts to conversation dynamics

4. **Theoretical Grounding**: Framework prevents drift
   - Concepts constrain the possibility space
   - Analyst coding reinforces alignment

### Scaling to CES (Phase 4)

This architecture enables:
- Demographics -> Agent personas (survey responses generate agents)
- Population-level social dynamics
- Emergent collective behavior patterns